### Query Enhancement – Query Expansion Techniques

In a RAG pipeline, the quality of the query sent to the retriever determines how good the retrieved context is — and therefore, how accurate the LLM’s final answer will be.

That’s where Query Expansion / Enhancement comes in.

#### 🎯 What is Query Enhancement?
Query enhancement refers to techniques used to improve or reformulate the user query to retrieve better, more relevant documents from the knowledge base.
It is especially useful when:

- The original query is short, ambiguous, or under-specified
- You want to broaden the scope to catch synonyms, related phrases, or spelling variants

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

In [2]:
## step1 : Load and split the dataset
loader = TextLoader("langchain_crewai_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)


In [3]:
chunks

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and other knowledge sources. (v1)'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse chains using st

In [4]:
### step 2: Vector Store
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

## step 3:MMR Retriever
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":5})
retriever


e:\RAGUdemy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029BEC799A90>, search_type='mmr', search_kwargs={'k': 5})

In [5]:
## step 4 : LLM and Prompt

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm=init_chat_model("openai:o4-mini")
llm


ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000029BF1766510>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000029BF1766F90>, root_client=<openai.OpenAI object at 0x0000029BF058B620>, root_async_client=<openai.AsyncOpenAI object at 0x0000029BF1766CF0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [6]:
# Query expansion
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.

Original query: "{query}"

Expanded query:
""")

query_expansion_chain=query_expansion_prompt| llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.\n\nOriginal query: "{query}"\n\nExpanded query:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000029BF1766510>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000029BF1766F90>, root_client=<openai.OpenAI object at 0x0000029BF058B620>, root_async_client=<openai.AsyncOpenAI object at 0x0000029BF1766CF0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [7]:
query_expansion_chain.invoke({"query":"Langchain memory"})

'Expanded query:\n\n("LangChain memory" OR "LangChain memory management" OR "LangChain memory module" OR "persistent conversation memory" OR "embeddings-based memory" OR "stateful chatbots" OR "session context persistence" OR "RAG memory store" OR "LLM context window" OR "prompt memory" OR "memory retriever")  \nAND  \n("VectorStoreMemory" OR "ConversationBufferMemory" OR "ConversationSummaryMemory" OR "RedisMemory" OR "SQLMemory" OR "MemoryChain" OR "MemoryRouter")  \nAND  \n(FAISS OR Chroma OR Pinecone OR Weaviate OR Milvus OR "vector embedding store" OR "document chunking" OR "knowledge retrieval")'

In [8]:
# RAG answering prompt
answer_prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

document_chain=create_stuff_documents_chain(llm=llm,prompt=answer_prompt)

In [9]:
# Step 5: Full RAG pipeline with query expansion
rag_pipeline = (
    RunnableMap({
        "input": lambda x: x["input"],
        "context": lambda x: retriever.invoke(query_expansion_chain.invoke({"query": x["input"]}))
    })
    | document_chain
)

In [10]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Here’s an expanded search query that adds synonyms, technical terms and useful context for better recall of LangChain’s memory features:

“LangChain memory support” OR “LangChain memory types” OR “LangChain memory modules” OR “LangChain memory classes”  
AND (“ConversationBufferMemory” OR “ConversationSummaryMemory” OR “CombinedMemory” OR “DynamicMemory” OR “ConversationTokenBufferMemory”)  
OR (“short-term memory” OR “long-term memory” OR “ephemeral memory” OR “session state” OR “context window”)  
OR (“persistent memory” OR “stateful memory store” OR “memory retriever”)  
OR (“vector store memory” OR “embedding store” OR “semantic memory” OR “RAG”)  
OR (“Chroma” OR “FAISS” OR “Pinecone” OR “Weaviate” OR “Redis” OR “SQLite” OR “PostgreSQL” OR “MongoDB”)  
OR (“memory API” OR “memory variable” OR “memory backend” OR “cache” OR “in-memory” OR “file-based”)  
AND (“LangChain Python” OR “LLMChain” OR “chatbot context management” OR “agent state management”)
✅ Answer:
 LangChain currently

In [12]:
# Step 6: Run query
query = {"input": "CrewAI agents?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Expanded query:

("CrewAI agents" OR "Crew AI agents" OR "CrewAI bots" OR "Crew AI assistants" OR "autonomous crew agents" OR "AI-driven crew management assistants" OR "virtual crew members" OR "digital crew agents")  
AND  
("crew scheduling" OR "workforce management" OR "staff rostering" OR "resource allocation" OR "employee roster optimization")  
AND  
("multi-agent system" OR "autonomous agents" OR "distributed AI" OR "agent-based modeling")  
AND  
("machine learning" OR "reinforcement learning" OR "predictive analytics" OR "optimization algorithms" OR "real-time planning")  
AND  
("airline" OR "maritime" OR "hospitality" OR "logistics" OR "field service")
✅ Answer:
 CrewAI agents are semi-autonomous, role-specialized AI “workers” that team up in a predefined workflow to tackle complex, multi-step tasks.  Key points:  
1. Defined Roles  
   • Researcher – gathers data and insights  
   • Planner – lays out strategy, timelines, and dependencies  
   • Executor – carries out concr